In [1]:
import os 
import pickle
import numpy as np
import pandas as pd 
from glob import glob 
from sklearn.metrics import accuracy_score, classification_report
from scipy.spatial.distance import mahalanobis

In [2]:
def dtw_distance(x, y, dist='euclidean'):
    m, n = len(x), len(y)
    dtw = np.zeros((m+1, n+1))
    
    # Initialize the first row and column of the DTW matrix
    for i in range(1, m+1):
        dtw[i, 0] = float('inf')
    for j in range(1, n+1):
        dtw[0, j] = float('inf')
    dtw[0, 0] = 0
    
    # Compute the DTW matrix
    for i in range(1, m+1):
        for j in range(1, n+1):
            if dist == 'euclidean':
              cost = euclidean_distance(x[i-1], y[j-1])  # Distance measure between elements x[i-1] and y[j-1]
            if dist == 'mahalanobis':
              cost = mahalanobis_distance(x[i-1], y[j-1])
            dtw[i, j] = cost + min(dtw[i-1, j], dtw[i, j-1], dtw[i-1, j-1])
    
    return dtw[m, n]

def euclidean_distance(a, b):
    # Calculate the distance between two elements a and b
    # Modify this function to use an appropriate distance measure for your multivariate time series
    
    # For example, Euclidean distance can be used as follows:
    return np.linalg.norm(a - b)
    
    # Alternatively, you can use other distance measures like Manhattan distance or cosine similarity
    # pass

def mahalanobis_distance(a, b):
    # Calculate the Mahalanobis distance between two elements a and b
    # cov is the covariance matrix
    # cov = np.array([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.],
    #                 [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1.],])
    
    cov = np.array([[1., 1. , 0.5, 0.5, 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [1., 1. , 0.5, 0.5, 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0.5, 0.5, 1. , 1. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0.5, 0.5, 1. , 1. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0., 0. , 0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0., 0. , 0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0., 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0., 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0., 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0., 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 1. , 0.5, 0.5, 0.5, 0.5, 0.,  0.,  0.,  0.,  0.,  0.,],
                    [0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 1. , 0.5, 0.5, 0.5, 0.5, 0.,  0.,  0.,  0.,  0.,  0.,],
                    [0., 0., 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 1. , 1. , 0. , 0. ,0.5, 0.5, 0. , 0. , 0. , 0. ,],
                    [0., 0., 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 1. , 1. , 0. , 0. ,0.5, 0.5, 0. , 0. , 0. , 0. ,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 1. , 1. ,0. , 0. , 0.5, 0.5, 0. , 0. ,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 1. , 1. ,0. , 0. , 0.5, 0.5, 0. , 0. ,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. ,1. , 1. , 0. , 0. , 0.5, 0.5,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. ,1. , 1. , 0. , 0. , 0.5, 0.5,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0.,  0.,  1.,  1.,  0.,  0.,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0.,  0.,  1.,  1.,  0.,  0.,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,0.5, 0.5, 0. , 0. , 1. , 1. ,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,0.5, 0.5, 0. , 0. , 1. , 1. ,]])
    # Calculate the Mahalanobis distance using the scipy.spatial.distance.mahalanobis function
    return mahalanobis(a, b, cov)

In [3]:
# Example usage
x = np.array([[1, 3, 5], [2, 4, 6], [1, 2, 3]])
y = np.array([[1, 4, 7], [3, 5, 7], [1, 2, 3], [4, 5, 6]])
distance = dtw_distance(x, y)
print("DTW distance:", distance)

DTW distance: 9.164271207775299


In [4]:
x.shape

(3, 3)

---

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data_root = '/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/epoch_vids/CKIM/UTD-MHAD/'

In [7]:
with open(f"{data_root}skeleton_label_predictions.pickle", 'rb') as handle:
    pred_skeleton_data = pickle.load(handle)

In [8]:
with open(f"{data_root}skeleton_original_seq.pickle", 'rb') as handle:
    origin_skeleton_data = pickle.load(handle)

In [9]:
_, _, files = next(os.walk(f"{data_root}/videos"))


In [10]:

pred_files, target_files = {}, {}
for f in files:
  id = int(f.split('_')[1])
  if f[-17:] == 'original_skel.mp4':
    target_files[id] = f
  else:
    pred_files[id] = f.split('_m_')[1].split('_')[0]


---

In [11]:
pred_df = pd.DataFrame().from_records(pred_skeleton_data['label'])
pred_df.head()

,id,target,prediction
0,0,tennis swing,throw
1,1,tennis swing,draw circle(counter clockwise)
2,2,throw,draw circle(counter clockwise)
3,3,throw,draw circle(counter clockwise)
4,4,lunge,throw


In [12]:
pred_df['matching'] = pred_df['id'].map(pred_files)
pred_df.head()

,id,target,prediction,matching
0,0,tennis swing,throw,boxing
1,1,tennis swing,draw circle(counter clockwise),draw x
2,2,throw,draw circle(counter clockwise),draw x
3,3,throw,draw circle(counter clockwise),draw x
4,4,lunge,throw,boxing


---

In [13]:
# UTD-MHAD
mhad_super_dict = {
    'One-hand activities': ['swipe left', 'swipe right', 'wave', 'throw', 'knock', 'catch'],
    'Two-hand activities': ['clap', 'arm cross', 'arm curl', 'push'],
    'Drawing activities': ['draw x', 'draw circle(clockwise)', 'draw circle(counter clockwise)', 'draw triangle'],
    'Sports': ['basketball shoot', 'boxing', 'baseball swing', 'tennis serve', 'tennis swing'],
    'Activities involving legs': ['bowling', 'pickup & throw', 'jog', 'walk', 'sit to stand', 'stand to sit', 'lunge', 'squat']
}


---

In [14]:
super_class = {}
for i,j in mhad_super_dict.items():
    for c in j:
        super_class[c] = i 

In [15]:
super_class

{'swipe left': 'One-hand activities',
 'swipe right': 'One-hand activities',
 'wave': 'One-hand activities',
 'throw': 'One-hand activities',
 'knock': 'One-hand activities',
 'catch': 'One-hand activities',
 'clap': 'Two-hand activities',
 'arm cross': 'Two-hand activities',
 'arm curl': 'Two-hand activities',
 'push': 'Two-hand activities',
 'draw x': 'Drawing activities',
 'draw circle(clockwise)': 'Drawing activities',
 'draw circle(counter clockwise)': 'Drawing activities',
 'draw triangle': 'Drawing activities',
 'basketball shoot': 'Sports',
 'boxing': 'Sports',
 'baseball swing': 'Sports',
 'tennis serve': 'Sports',
 'tennis swing': 'Sports',
 'bowling': 'Activities involving legs',
 'pickup & throw': 'Activities involving legs',
 'jog': 'Activities involving legs',
 'walk': 'Activities involving legs',
 'sit to stand': 'Activities involving legs',
 'stand to sit': 'Activities involving legs',
 'lunge': 'Activities involving legs',
 'squat': 'Activities involving legs'}

In [16]:
pred_df['super_target'] = pred_df['target'].map(super_class)
pred_df['super_prediction'] = pred_df['prediction'].map(super_class)
pred_df['super_matching'] = pred_df['matching'].map(super_class)
pred_df.head()

,id,target,prediction,matching,super_target,super_prediction,super_matching
0,0,tennis swing,throw,boxing,Sports,One-hand activities,Sports
1,1,tennis swing,draw circle(counter clockwise),draw x,Sports,Drawing activities,Drawing activities
2,2,throw,draw circle(counter clockwise),draw x,One-hand activities,Drawing activities,Drawing activities
3,3,throw,draw circle(counter clockwise),draw x,One-hand activities,Drawing activities,Drawing activities
4,4,lunge,throw,boxing,Activities involving legs,One-hand activities,Sports


In [17]:
pred_df['correct'] = (pred_df['target'] == pred_df['prediction']).astype(int)
pred_df.head()

,id,target,prediction,matching,super_target,super_prediction,super_matching,correct
0,0,tennis swing,throw,boxing,Sports,One-hand activities,Sports,0
1,1,tennis swing,draw circle(counter clockwise),draw x,Sports,Drawing activities,Drawing activities,0
2,2,throw,draw circle(counter clockwise),draw x,One-hand activities,Drawing activities,Drawing activities,0
3,3,throw,draw circle(counter clockwise),draw x,One-hand activities,Drawing activities,Drawing activities,0
4,4,lunge,throw,boxing,Activities involving legs,One-hand activities,Sports,0


In [18]:
# super class accuracy 
correct_df = pred_df.loc[pred_df['correct'] == 1, :]
print(accuracy_score(y_true=correct_df['super_prediction'], y_pred=correct_df['super_matching']))
print(classification_report(y_true=correct_df['super_prediction'], y_pred=correct_df['super_matching'], zero_division=0))

0.8214285714285714
                           precision    recall  f1-score   support

Activities involving legs       0.67      1.00      0.80         6
       Drawing activities       0.94      1.00      0.97        17
      One-hand activities       0.00      0.00      0.00         2
                   Sports       0.00      0.00      0.00         3
      Two-hand activities       0.00      0.00      0.00         0

                 accuracy                           0.82        28
                macro avg       0.32      0.40      0.35        28
             weighted avg       0.72      0.82      0.76        28



In [19]:
# super class tendency - predictiion super class 
incorrect_df = pred_df.loc[pred_df['correct'] == 0, :]
print(accuracy_score(y_true=incorrect_df['super_prediction'][:-1], y_pred=incorrect_df['super_matching'][:-1]))
print(classification_report(y_true=incorrect_df['super_prediction'][:-1], y_pred=incorrect_df['super_matching'][:-1], zero_division=0))

0.34285714285714286
                           precision    recall  f1-score   support

Activities involving legs       0.08      1.00      0.14         1
       Drawing activities       0.79      0.65      0.71        17
      One-hand activities       0.00      0.00      0.00         7
                   Sports       0.00      0.00      0.00        10
      Two-hand activities       0.00      0.00      0.00         0

                 accuracy                           0.34        35
                macro avg       0.17      0.33      0.17        35
             weighted avg       0.38      0.34      0.35        35



In [20]:
# super class tendency - predictiion super class 
incorrect_df = pred_df.loc[pred_df['correct'] == 0, :]
print(accuracy_score(y_true=incorrect_df['super_target'][:-1], y_pred=incorrect_df['super_matching'][:-1]))
print(classification_report(y_true=incorrect_df['super_target'][:-1], y_pred=incorrect_df['super_matching'][:-1], zero_division=0))

0.42857142857142855
                           precision    recall  f1-score   support

Activities involving legs       0.77      0.91      0.83        11
       Drawing activities       0.00      0.00      0.00         0
      One-hand activities       0.00      0.00      0.00        12
                   Sports       0.83      0.42      0.56        12
      Two-hand activities       0.00      0.00      0.00         0

                 accuracy                           0.43        35
                macro avg       0.32      0.27      0.28        35
             weighted avg       0.53      0.43      0.45        35



In [21]:
# overall interpretability 
print(accuracy_score(y_true=pred_df['super_prediction'][:-1], y_pred=pred_df['super_matching'][:-1]))
print(classification_report(y_true=pred_df['super_prediction'][:-1], y_pred=pred_df['super_matching'][:-1], zero_division=0))

0.5555555555555556
                           precision    recall  f1-score   support

Activities involving legs       0.32      1.00      0.48         7
       Drawing activities       0.85      0.82      0.84        34
      One-hand activities       0.00      0.00      0.00         8
                   Sports       0.00      0.00      0.00        14
      Two-hand activities       0.00      0.00      0.00         0

                 accuracy                           0.56        63
                macro avg       0.23      0.36      0.26        63
             weighted avg       0.49      0.56      0.50        63



---

In [22]:
def apply_func(r):
  i = r['id']
  return dtw_distance(origin_skeleton_data[r['matching']], pred_skeleton_data['data'][i])

In [23]:
pred_df['dtw_distance'] = pred_df.apply(lambda r: apply_func(r), axis=1)
pred_df.head()

,id,target,prediction,matching,super_target,super_prediction,super_matching,correct,dtw_distance
0,0,tennis swing,throw,boxing,Sports,One-hand activities,Sports,0,7.073498
1,1,tennis swing,draw circle(counter clockwise),draw x,Sports,Drawing activities,Drawing activities,0,9.247147
2,2,throw,draw circle(counter clockwise),draw x,One-hand activities,Drawing activities,Drawing activities,0,9.010388
3,3,throw,draw circle(counter clockwise),draw x,One-hand activities,Drawing activities,Drawing activities,0,8.695119
4,4,lunge,throw,boxing,Activities involving legs,One-hand activities,Sports,0,9.518208


In [24]:
def apply_func(r):
  i = r['id']
  return dtw_distance(origin_skeleton_data[r['matching']], pred_skeleton_data['data'][i], dist='mahalanobis')

pred_df['mahalanobis_distance'] = pred_df.apply(lambda r: apply_func(r), axis=1)
pred_df.head()

,id,target,prediction,matching,super_target,super_prediction,super_matching,correct,dtw_distance,mahalanobis_distance
0,0,tennis swing,throw,boxing,Sports,One-hand activities,Sports,0,7.073498,7.613114
1,1,tennis swing,draw circle(counter clockwise),draw x,Sports,Drawing activities,Drawing activities,0,9.247147,10.827675
2,2,throw,draw circle(counter clockwise),draw x,One-hand activities,Drawing activities,Drawing activities,0,9.010388,10.512138
3,3,throw,draw circle(counter clockwise),draw x,One-hand activities,Drawing activities,Drawing activities,0,8.695119,10.289783
4,4,lunge,throw,boxing,Activities involving legs,One-hand activities,Sports,0,9.518208,11.138175


In [25]:
pred_df.loc[pred_df['correct'] == 0, :]['dtw_distance'][:-1].mean(), pred_df.loc[pred_df['correct'] == 0, :]['mahalanobis_distance'][:-1].mean()

(10.711311406373756, 12.90428417056878)

In [26]:
pred_df.loc[pred_df['correct'] == 0, :]['dtw_distance'][:-1].std(), pred_df.loc[pred_df['correct'] == 0, :]['mahalanobis_distance'][:-1].std()

(2.6063471662532716, 3.609105280010131)

In [27]:
pred_df.loc[pred_df['correct'] == 1, :]['dtw_distance'].mean(), pred_df.loc[pred_df['correct'] == 1, :]['mahalanobis_distance'].mean()

(11.051813272211845, 13.771628513156283)

In [28]:
pred_df.loc[pred_df['correct'] == 1, :]['dtw_distance'].std(), pred_df.loc[pred_df['correct'] == 1, :]['mahalanobis_distance'].std()

(4.3570007619708235, 6.614170164134862)

In [29]:
pred_df.loc[pred_df['correct'] == 1, :]

,id,target,prediction,matching,super_target,super_prediction,super_matching,correct,dtw_distance,mahalanobis_distance
5,5,draw circle(counter clockwise),draw circle(counter clockwise),draw x,Drawing activities,Drawing activities,Drawing activities,1,8.633956,10.246049
6,6,throw,throw,draw x,One-hand activities,One-hand activities,Drawing activities,1,9.833749,12.565101
7,7,draw circle(counter clockwise),draw circle(counter clockwise),draw x,Drawing activities,Drawing activities,Drawing activities,1,7.999544,8.622921
8,8,draw circle(counter clockwise),draw circle(counter clockwise),draw x,Drawing activities,Drawing activities,Drawing activities,1,7.866846,8.322384
9,9,draw circle(counter clockwise),draw circle(counter clockwise),draw x,Drawing activities,Drawing activities,Drawing activities,1,8.129501,8.962349
13,13,tennis swing,tennis swing,bowling,Sports,Sports,Activities involving legs,1,12.787348,18.394105
16,16,draw circle(counter clockwise),draw circle(counter clockwise),draw x,Drawing activities,Drawing activities,Drawing activities,1,7.888497,8.312308
17,17,draw circle(counter clockwise),draw circle(counter clockwise),draw x,Drawing activities,Drawing activities,Drawing activities,1,8.093648,9.255118
19,19,draw circle(counter clockwise),draw circle(counter clockwise),draw x,Drawing activities,Drawing activities,Drawing activities,1,8.314177,9.648435
21,21,lunge,lunge,squat,Activities involving legs,Activities involving legs,Activities involving legs,1,20.439095,26.793852
